In [32]:
from PIL import Image, ImageDraw, ImageFont
import os
import cv2
import numpy as np



## Directorys Laden

In [14]:
# Pfade zu den Bildverzeichnissen
image_filenames_selected = "Vis/SelectedforVis/"
image_path = "Dataset_new/images/"
mask_path = "Dataset_new/masks/"
opencv_path = "results/opencv/"
sd_path = "results/sd/"
lama_path = "results/lama/"


In [15]:
# Liste der Dateinamen (ohne Verzeichnisse)
image_filenames = [
    "ADE_frame_00000021.jpg",
    "ADE_frame_00000042.jpg",
    "ADE_frame_00000045.jpg",
    "ADE_frame_00000094.jpg",
    "ADE_frame_00000105.jpg",
    "ADE_frame_00000154.jpg",
    "ADE_frame_00000166.jpg",
    "ADE_train_00000003.jpg",
    "ADE_train_00000651.jpg",
    "ADE_train_00000655.jpg",
    "ADE_train_00000658.jpg",
    "ADE_train_00000667.jpg",
    "ADE_train_00001222.jpg",
    "ADE_train_00001489.jpg",
    "ADE_train_00001490.jpg",
    "ADE_train_00001492.jpg",
    "ADE_train_00001494.jpg",
    "ADE_train_00001551.jpg",
    "ADE_train_00001552.jpg",
    "ADE_train_00001554.jpg",
    "ADE_train_00001558.jpg",
    "ADE_train_00001561.jpg",
    "ADE_train_00001607.jpg",
    "ADE_train_00001633.jpg",
    "ADE_train_00001634.jpg",
    "ADE_train_00001639.jpg",
    "ADE_train_00001646.jpg",
    "ADE_train_00001657.jpg",
    "ADE_train_00001707.jpg",
    "ADE_train_00005392.jpg",
    "ADE_train_00009499.jpg",
    "ADE_train_00012229.jpg",
    "ADE_train_00015288.jpg",
    "ADE_train_00022632.jpg",
    "ADE_train_00022636.jpg",
    "ADE_train_00022703.jpg",
    "ADE_train_00024787.jpg",
    "ADE_train_00025023.jpg"
]



## Original - Mask - Lama Laden

In [16]:
# Zielverzeichnis für die kombinierten Bilder
output_dir = "Vis/Lama/combined_images/"
os.makedirs(output_dir, exist_ok=True)


In [17]:
for filename in image_filenames:
    try:
        # Bilder laden
        original_img = Image.open(os.path.join(image_path, filename))
        mask_img = Image.open(os.path.join(mask_path, filename))
        lama_img = Image.open(os.path.join(lama_path, filename))

        # Alle Bilder auf die gleiche Größe skalieren
        width, height = original_img.size
        mask_img = mask_img.resize((width, height))
        lama_img = lama_img.resize((width, height))
        
        

        # Kombinierte Breite und Höhe berechnen
        combined_width = width * 3
        combined_height = height

        # Neues Bild erstellen
        combined_img = Image.new("RGB", (combined_width, combined_height))

        # Einzelbilder nebeneinander einfügen
        combined_img.paste(original_img, (0, 0))
        combined_img.paste(mask_img, (width, 0))
        combined_img.paste(lama_img, (width * 2, 0))
        
    

        # Kombiniertes Bild speichern
        output_path = os.path.join(output_dir, filename)
        combined_img.save(output_path)

        print(f"Kombiniertes Bild gespeichert: {output_path}")
    except Exception as e:
        print(f"Fehler bei {filename}: {e}")

Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_frame_00000021.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_frame_00000042.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_frame_00000045.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_frame_00000094.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_frame_00000105.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_frame_00000154.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_frame_00000166.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_train_00000003.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_train_00000651.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_train_00000655.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_train_00000658.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_images/ADE_train_00000667.jpg
Kombiniertes Bild gespeichert: Vis/Lama/combined_ima

## Alle Bilder Laden mit Titel

In [18]:
# Zielverzeichnis für die kombinierten Bilder
output_dir_2 = "Vis/combined_images/"
os.makedirs(output_dir_2, exist_ok=True)

In [48]:
def ensure_rgb(img_cv):
    """Stellt sicher, dass das Bild im RGB-Format vorliegt."""
    if len(img_cv.shape) == 2:  # Graustufenbild
        img_cv = cv2.cvtColor(img_cv, cv2.COLOR_GRAY2RGB)
    return img_cv



In [52]:
def add_title(img, text, width, font_size=20):
    """Fügt einen Titel über dem Bild hinzu."""
    # Platz für Titel hinzufügen
    new_height = img.height + font_size + 10  # 10 px Abstand
    new_img = Image.new("RGB", (width, new_height), "white")
    new_img.paste(img, (0, font_size + 10))
    
    # Text zeichnen
    draw = ImageDraw.Draw(new_img)
    try:
        # Standard-Schriftart verwenden
        font = ImageFont.load_default()
    except:
        font = None

    # Textbreite und Höhe berechnen
    if font:
        if hasattr(font, "getbbox"):  # Neuere Pillow-Versionen
            bbox = font.getbbox(text)
            text_width = bbox[2] - bbox[0]
            text_height = bbox[3] - bbox[1]
        else:  # Ältere Pillow-Versionen
            text_width, text_height = font.getsize(text)
    else:
        text_width = len(text) * font_size // 2  # Schätzung für Breite ohne Font
        text_height = font_size  # Schätzung der Höhe

    # Text zentrieren
    text_x = (width - text_width) // 2
    draw.text((text_x, 5), text, fill="black", font=font)
    return new_img

In [53]:
for filename in image_filenames:
    try:
        # Bilder laden
        original_img = Image.open(os.path.join(image_path, filename))
        mask_img = Image.open(os.path.join(mask_path, filename))
        opencv_img = Image.open(os.path.join(opencv_path, filename))
        sd_img = Image.open(os.path.join(sd_path, filename))
        lama_img = Image.open(os.path.join(lama_path, filename))

        # Alle Bilder auf die gleiche Größe skalieren
        width, height = original_img.size
        mask_img = mask_img.resize((width, height))
        opencv_img = opencv_img.resize((width, height))
        sd_img = sd_img.resize((width, height))
        lama_img = lama_img.resize((width, height))

        # Titel hinzufügen
        original_img = add_title(original_img, "Original Image", width)
        mask_img = add_title(mask_img, "Mask Image", width)
        opencv_img = add_title(opencv_img, "OpenCV Inpainting", width)
        sd_img = add_title(sd_img, "SD Inpainting", width)
        lama_img = add_title(lama_img, "LaMa Inpainting", width)

        # Kombinierte Breite und Höhe berechnen
        combined_width = width * 5  # Für 5 Bilder
        combined_height = original_img.height  # Höhe inkl. Titel

        # Neues Bild erstellen
        combined_img = Image.new("RGB", (combined_width, combined_height), "white")

        # Einzelbilder nebeneinander einfügen
        combined_img.paste(original_img, (0, 0))
        combined_img.paste(mask_img, (width, 0))
        combined_img.paste(opencv_img, (width * 2, 0))
        combined_img.paste(sd_img, (width * 3, 0))
        combined_img.paste(lama_img, (width * 4, 0))

        # Kombiniertes Bild speichern
        output_path = os.path.join(output_dir_2, filename)
        combined_img.save(output_path)

        print(f"Kombiniertes Bild gespeichert: {output_path}")
    except Exception as e:
        print(f"Fehler bei {filename}: {e}")

Kombiniertes Bild gespeichert: Vis/combined_images/ADE_frame_00000021.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_frame_00000042.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_frame_00000045.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_frame_00000094.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_frame_00000105.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_frame_00000154.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_frame_00000166.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_train_00000003.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_train_00000651.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_train_00000655.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_train_00000658.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_train_00000667.jpg
Kombiniertes Bild gespeichert: Vis/combined_images/ADE_train_00001222.jpg
Kombiniertes Bild gespeichert: Vis/com

## Alle Bilder Laden ohne Titel

In [21]:
# Zielverzeichnis für die kombinierten Bilder
output_dir_3= "Vis/combined_images_without_titel/"
os.makedirs(output_dir_3, exist_ok=True)

In [22]:
for filename in image_filenames:
    try:
        # Bilder laden
        original_img = Image.open(os.path.join(image_path, filename))
        mask_img = Image.open(os.path.join(mask_path, filename))
        opencv_img = Image.open(os.path.join(opencv_path, filename))
        sd_img = Image.open(os.path.join(sd_path, filename))
        lama_img = Image.open(os.path.join(lama_path, filename))

        # Alle Bilder auf die gleiche Größe skalieren
        width, height = original_img.size
        mask_img = mask_img.resize((width, height))
        opencv_img = opencv_img.resize((width, height))
        sd_img = sd_img.resize((width, height))
        lama_img = lama_img.resize((width, height))

        # Kombinierte Breite und Höhe berechnen
        combined_width = width * 5  # Für 5 Bilder
        combined_height = original_img.height  # Höhe inkl. Titel

        # Neues Bild erstellen
        combined_img = Image.new("RGB", (combined_width, combined_height), "white")

        # Einzelbilder nebeneinander einfügen
        combined_img.paste(original_img, (0, 0))
        combined_img.paste(mask_img, (width, 0))
        combined_img.paste(opencv_img, (width * 2, 0))
        combined_img.paste(sd_img, (width * 3, 0))
        combined_img.paste(lama_img, (width * 4, 0))

        # Kombiniertes Bild speichern
        output_path3 = os.path.join(output_dir_3, filename)
        combined_img.save(output_path3)

        print(f"Kombiniertes Bild gespeichert: {output_path3}")
    except Exception as e:
        print(f"Fehler bei {filename}: {e}")

Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_frame_00000021.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_frame_00000042.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_frame_00000045.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_frame_00000094.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_frame_00000105.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_frame_00000154.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_frame_00000166.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_train_00000003.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_train_00000651.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_train_00000655.jpg
Kombiniertes Bild gespeichert: Vis/combined_images_without_titel/ADE_train_00000658.jpg
Kombiniertes Bild gespeichert: V

## Collagen ohne Titel

In [23]:
# Pfad zu den Bildern
input_dir1 = "Vis/SelectedforVis/"
output_path1 = "Vis/Collage/combined_vertical_image.jpg"  # Ausgabe-Datei

input_dir2 = "Vis/selected_2/"
output_path2 = "Vis/Collage/selected2.jpg"  # Ausgabe-Datei

input_dir3 = "Vis/selected_3/"
output_path3 = "Vis/Collage/selected3.jpg"  # Ausgabe-Datei

In [24]:
from PIL import Image
import os

def create_collage(input_dir, output_path):
    # Alle Bilddateien aus dem Verzeichnis abrufen
    image_filenames = [f for f in os.listdir(input_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # Sortiere die Bilder für Konsistenz (optional)
    image_filenames.sort()

    # Liste, um geöffnete Bilder zu speichern
    images = []

    # Alle Bilder laden und auf gleiche Größe skalieren
    for filename in image_filenames:
        img_path = os.path.join(input_dir, filename)
        try:
            img = Image.open(img_path)

            # Hier skalieren wir jedes Bild auf die Größe des größten Bildes
            if images:
                max_width = max((img.width for img in images), default=img.width)
                max_height = max((img.height for img in images), default=img.height)
            else:
                max_width, max_height = img.width, img.height

            img = img.resize((max_width, max_height))  # Skalieren auf die Größe des größten Bildes
            images.append(img)
        except Exception as e:
            print(f"Fehler beim Laden von {filename}: {e}")

    # Überprüfen, ob Bilder geladen wurden
    if not images:
        print("Keine gültigen Bilder gefunden!")
    else:
        # Breite und Höhe des kombinierten Bildes berechnen
        total_width = max(img.width for img in images)
        total_height = sum(img.height for img in images)

        # Neues Bild erstellen
        combined_img = Image.new("RGB", (total_width, total_height), "white")

        # Bilder untereinander anordnen
        y_offset = 0
        for img in images:
            combined_img.paste(img, (0, y_offset))
            y_offset += img.height

        # Kombiniertes Bild speichern
        combined_img.save(output_path)
        print(f"Kombiniertes Bild gespeichert unter: {output_path}")

# Definiere die Verzeichnisse und Ausgabepfade für alle 3 Collagen
collage_info = [
    ("Vis/SelectedforVis/", "Vis/Collage/combined_vertical_image_without.jpg"),
    ("Vis/selected_2/", "Vis/Collage/selected2_without.jpg"),
    ("Vis/selected_3/", "Vis/Collage/selected3_without.jpg")
]

# Erstelle für jedes Set von Bildern eine Collage
for input_dir, output_path in collage_info:
    create_collage(input_dir, output_path)


Kombiniertes Bild gespeichert unter: Vis/Collage/combined_vertical_image_withtitle.jpg
Kombiniertes Bild gespeichert unter: Vis/Collage/selected2_withtitle.jpg
Kombiniertes Bild gespeichert unter: Vis/Collage/selected3_withtitle.jpg
